In [1]:
import torch
from torch.utils.data import DataLoader

import numpy as np

from rpn.build_rpn import RPN_Model
from sam.build_sam import SAM_Model
from transformers import pipeline

from data_builder.build_dataset import PlanetscopeDataset

import json

from eval_utils import filter_boxes, calculate_iou, calculate_precision_recall

import pandas as pd

/home/aghosh57/.conda/envs/dl4cv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### Read and Print the evaluation config from the JSON file

In [2]:
with open('eval_config.json', 'r') as f:
    eval_config = json.load(f)

In [3]:
print("Evaluation Configuration")
print(json.dumps(eval_config, indent=1))

Evaluation Configuration
{
 "DATASET_PATH": "/scratch/aghosh57/SAT-SAM(Dataset)/ps_france/all_dataset/",
 "REGION": "France-1.14_run4",
 "MODEL": "SAT-SAM",
 "NMS_THRESHOLD": 0.9,
 "PRED_CONFIDENCE_THRESHOLD": 0.6,
 "ENSEMBLE_BOX_THRESHOLD": 0.5,
 "ENSEMBLE_BOX_BETA": 0.4,
 "RPN_MODEL_PATH": "rpn/checkpoint/rpn_model_1.14_Fr_Rw.pth",
 "SAM_MODEL_PATH": "sam/checkpoint/sam_vit_l_0b3195.pth"
}


#### Set the device and load the dataset for evaluation

In [4]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
dataset = PlanetscopeDataset(eval_config['DATASET_PATH'], train=False)

#### Load the proper model to be used for evaluation

In [5]:
if eval_config['MODEL'] == 'SAT-SAM':
    rpn_model = RPN_Model(eval_config['RPN_MODEL_PATH'], 2, device)
    sam_model = SAM_Model(eval_config['SAM_MODEL_PATH'], 'large', device)
elif eval_config['MODEL'] == 'SAM':
    vanilla_sam_model = pipeline("mask-generation", model="facebook/sam-vit-large", device=device)
elif eval_config['MODEL'] == 'MASKRCNN':
    maskrcnn_model = RPN_Model(eval_config['RPN_MODEL_PATH'], 2, device) #Load the Pre-Trained MaskRCNN model
    # maskrcnn_model = RPN_Model(None, 2, device)   #Load the Vanilla MaskRCNN model

In [6]:
results = pd.DataFrame(columns=['parcel_id', 'gt_mask_ct', 'pred_mask_ct', 'mean_iou', 'p_50', 'r_50', 'p_70', 'r_70', 'p_90', 'r_90'])

for i, (sam_image, rpn_image, target, ensemble)  in enumerate(dataset): 
    try:
        if eval_config['MODEL'] == 'SAT-SAM':
            print('SAT-SAM')
            rpn_image = rpn_image.squeeze(0).to(device)  
            predictions = rpn_model.predict(rpn_image)
            predictions = rpn_model.postprocess(predictions, nms_threshold=eval_config['NMS_THRESHOLD'], score_threshold=eval_config['PRED_CONFIDENCE_THRESHOLD'])

            filtered_predictions = filter_boxes(predictions, ensemble, eval_config['ENSEMBLE_BOX_BETA'], eval_config['ENSEMBLE_BOX_THRESHOLD'])

            low_res_masks, iou_predictions = sam_model.predict(sam_image, filtered_predictions)
            high_res_masks = sam_model.postprocess(low_res_masks, tuple(sam_image.size))
            pred_masks = high_res_masks.squeeze().cpu().numpy()
        
        elif eval_config['MODEL'] == 'SAM':
            print('SAM')
            outputs = vanilla_sam_model(sam_image, points_per_batch=32)
            pred_masks = outputs["masks"]
        
        elif eval_config['MODEL'] == 'MASKRCNN':
            print('MASKRCNN')
            rpn_image = rpn_image.squeeze(0).to(device)
            predictions = maskrcnn_model.predict(rpn_image)
            predictions = maskrcnn_model.postprocess(predictions, nms_threshold=eval_config['NMS_THRESHOLD'], score_threshold=eval_config['PRED_CONFIDENCE_THRESHOLD'])
            pred_masks = predictions['masks']

        iou_score, iou_matrix = calculate_iou(target_masks=np.array(target['masks']), predicted_masks=np.array(pred_masks))
        
        p_50, r_50 = calculate_precision_recall(iou_matrix, len(pred_masks), len(target['masks']), threshold=0.5)
        p_70, r_70 = calculate_precision_recall(iou_matrix, len(pred_masks), len(target['masks']), threshold=0.7)
        p_90, r_90 = calculate_precision_recall(iou_matrix, len(pred_masks), len(target['masks']), threshold=0.9)
        
        # Round off the values to 2 decimal places
        iou_score = round(iou_score, 2)
        p_50, r_50 = round(p_50, 2), round(r_50, 2)
        p_70, r_70 = round(p_70, 2), round(r_70, 2)
        p_90, r_90 = round(p_90, 2), round(r_90, 2)

        print("Image Id: ", i, " Average IoU Score: ", iou_score)
        print("IoU Matrix: ", iou_matrix)
        print("Precision: ", p_50, p_70, p_90)
        print("Recall: ", r_50, r_70, r_90)

        results.loc[i] = [i, len(target['masks']), len(pred_masks), iou_score, p_50, r_50, p_70, r_70, p_90, r_90]
    except:
        print("Error in image: ", i)
        continue

SAT-SAM
num_target_masks:  9
num_predicted_masks:  12
row_ind:  [0 1 2 3 4 5 6 7 8]
col_ind:  [ 0  7  4  1  5  2 10  6  3]
Image Id:  0  Average IoU Score:  0.55
IoU Matrix:  [0.80674721 0.         0.50371538 0.82598413 0.79603707 0.57687148
 0.07661996 0.4845585  0.83936658]
Precision:  0.5 0.33 0.0
Recall:  0.67 0.44 0.0
SAT-SAM
num_target_masks:  10
num_predicted_masks:  22
row_ind:  [0 1 2 3 4 5 6 7 8 9]
col_ind:  [ 3 14 19  5 21  0  4 15  1  2]
Image Id:  1  Average IoU Score:  0.65
IoU Matrix:  [0.85152634 0.6188343  0.34413548 0.4479556  0.57560301 0.89169646
 0.82753165 0.40227048 0.85880547 0.63676198]
Precision:  0.32 0.18 0.0
Recall:  0.7 0.4 0.0
SAT-SAM
num_target_masks:  12
num_predicted_masks:  14
row_ind:  [ 0  1  2  3  4  5  6  7  8  9 10 11]
col_ind:  [ 5  3  9  2 11  4 10  8  7  6  1  0]
Image Id:  2  Average IoU Score:  0.24
IoU Matrix:  [8.18273092e-02 8.07804117e-01 0.00000000e+00 4.99393593e-04
 1.93641618e-01 2.87028519e-01 0.00000000e+00 0.00000000e+00
 0.000000

In [ ]:
# Input: dataframe, model name, dataset name
# Output: csv file

def save_csv(df, model_name, dataset_name):
    df.to_csv('{}_{}.csv'.format(model_name, dataset_name), index=False)

save_csv(results, eval_config['MODEL'], eval_config['REGION'])